<a href="https://colab.research.google.com/github/sheddyranking/Dog_Breeds_Classification/blob/main/Classifying_Dog_Breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UNZIP THE FILES**

In [2]:
#!unzip "/content/drive/MyDrive/ Classifying_Dogs_Breeds/dog-breed-identification.zip" -d "/content/drive/MyDrive/ Classifying_Dogs_Breeds/" 

# Using Transfer Learning and TensorFlow to Classify Different Dog Breeds 🐶

## Problem
Identifying the breed of a dog when given an image of a dog. for example when sitting at a stadium and i take a photo of a dog, i want to know what breed it is.

## Data
The data that we are using is from kaggle dogs breeds identification competition https://www.kaggle.com/competitions/dog-breed-identification/data

## Evaluation
The Evaluation is to submit a file with predictable probabilities for each Dog breed for each test image. https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

## Features
some information about the data:
* we are dealing with images (unstructured data) so it is probably best we use Deep Learning/Transfer Learning.
* There 120 Breeds of Dogs(there are 120 different classes)
* There are around 10,000+ images in the training set (this images have labels)
* There are around 10,000+ images in the testing set (this images do not have labels because we will want to predict them).


# Get our workspace ready
* Import TensorFlow 
* Import TensorFlow Hub
* Make sure we are using a GPU

In [1]:
## import the necessary libraries

import tensorflow as tf
import tensorflow_hub as hub

print("tf-version:",tf.__version__)
print("tfhub-version:",hub.__version__)

##check for GPU availability
print("GPU", "Available" if tf.config.list_physical_devices("GPU") else "not available") 

tf-version: 2.8.2
tfhub-version: 0.12.0
GPU Available
